# Implementação 1: Modelo de Rede Neural de Recorrência

Passo 1: Configuração do Ambiente no Google Colab

In [ ]:
# Importar bibliotecas necessárias
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, LSTM
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
print("Bibliotecas importadas com sucesso!")

Bibliotecas importadas com sucesso!


Explicação
numpy: Para operações numéricas.

*   numpy: Para operações numéricas
*   tensorflow.keras: A API de alto nível para construir e treinar modelos de deep learning
*   Embedding: Camada que transforma palavras (índices numéricos) em vetores densos.
*   SimpleRNN: A camada de Rede Neural Recorrente mais básica.
*   Dense: Camada neural comum (fully connected layer).
*   Tokenizer: Para converter texto em sequências de números.
*   pad_sequences: Para garantir que todas as sequências de entrada tenham o mesmo comprimento.

Passo 2: Preparação do Conjunto de Dados

In [ ]:
# 1. Definir o conjunto de treinamento de dados (frases e rótulos)
textos_treinamento = [
    "eu gosto de programar em python",
    "python é uma linguagem poderosa",
    "programação divertida em python",
    "aprenda python e seja feliz",
    "gosto de aprender coisas novas"
]
print(f"Textos de treinamento: {textos_treinamento}")

# 2. Inicializar o tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(textos_treinamento)

# 3. Converter textos para sequências de números
sequencias = tokenizer.texts_to_sequences(textos_treinamento)
print(f"Vocabulário (palavra: índice): {tokenizer.word_index}")
print(f"Sequências numéricas dos textos: {sequencias}")

# 4. Calcular o tamanho do vocabulário (+1 para incluir o 0 de padding)
total_palavras = len(tokenizer.word_index) + 1
print(f"Tamanho total do vocabulário: {total_palavras}")

# 5. Preparar Entradas (X) e Saídas (Y) para a previsão da próxima palavra
# A entrada (X) será uma sequência de palavras, e a saída (Y) será a palavra seguinte.
max_comprimento = max(len(seq) for seq in sequencias)
print(f"Comprimento máximo das sequências antes do padding: {max_comprimento}")

entradas_X = []
saidas_Y = []

for seq in sequencias:
    for i in range(1, len(seq)):
        entradas_X.append(seq[:i]) # A sequência até a palavra atual
        saidas_Y.append(seq[i])   # A próxima palavra

print(f"Exemplo de entradas X (parcial): {entradas_X[:5]}")
print(f"Exemplo de saídas Y (parcial): {saidas_Y[:5]}")

# 6. Padronizar o comprimento das sequências de entrada
# Todas as sequências de entrada precisam ter o mesmo comprimento para a RNN.
# Usamos max_len-1 porque a saída, que é a última palavra, pad_sequences sempre terá 1 palavra a menos.
entradas_X_padded = pad_sequences(entradas_X, maxlen=max_comprimento - 1, padding='pre')
print(f"Formato final das entradas (X): {entradas_X_padded.shape}")

# 7. Converter as saídas para o formato one-hot encoding
# Isso é necessário para a camada de saída do RNN (softmax)
saidas_Y_one_hot = tf.keras.utils.to_categorical(saidas_Y, num_classes=total_palavras)
print(f"Formato final das saídas (Y): {saidas_Y_one_hot.shape}")

Textos de treinamento: ['eu gosto de programar em python', 'python é uma linguagem poderosa', 'programação divertida em python', 'aprenda python e seja feliz', 'gosto de aprender coisas novas']
Vocabulário (palavra: índice): {'python': 1, 'gosto': 2, 'de': 3, 'em': 4, 'eu': 5, 'programar': 6, 'é': 7, 'uma': 8, 'linguagem': 9, 'poderosa': 10, 'programação': 11, 'divertida': 12, 'aprenda': 13, 'e': 14, 'seja': 15, 'feliz': 16, 'aprender': 17, 'coisas': 18, 'novas': 19}
Sequências numéricas dos textos: [[5, 2, 3, 6, 4, 1], [1, 7, 8, 9, 10], [11, 12, 4, 1], [13, 1, 14, 15, 16], [2, 3, 17, 18, 19]]
Tamanho total do vocabulário: 20
Comprimento máximo das sequências antes do padding: 6
Exemplo de entradas X (parcial): [[5], [5, 2], [5, 2, 3], [5, 2, 3, 6], [5, 2, 3, 6, 4]]
Exemplo de saídas Y (parcial): [2, 3, 6, 4, 1]
Formato final das entradas (X): (20, 5)
Formato final das saídas (Y): (20, 20)


Explicação:
*   Geracão de Pares (Sequência --> Próxima Palavra) para treinar a RNN a prever a próxima palavra, transformamos cada frase em múltiplos pares de (sequência parcial, próxima palavra).
*   pad_sequences: É vital. Como as sequências parciais têm comprimentos variados, pad_sequences preenche (com zeros, por padrão) as sequências mais curtas para que todas tenham o mesmo maxlen. O 'padding='pre' significa que os zeros são adicionados no início.
*   to_categorical: A camada de saída da RNN (com softmax) produz uma probablidade para cada palavra no vocabulário. to_categorical converte o índice da palavra real em um vetor, onde apenas a posição da palavra correta é 1 e o resto é 0, chamado de "one-hot encoding".

Passo 3: Construção do Modelo RNN

In [ ]:
# 1. Definindo o modelo
modelo_rnn = Sequential()

# 2. Camada de Embedding:
# total_palavras: tamanho do vocabulário
# 10: dimensão do vetor de embedding (quantas características queremos para cada palavra)
# input_length: comprimento padronizado das sequências de entrada (maxlen - 1)
modelo_rnn.add(Embedding(total_palavras, 10, input_length=entradas_X_padded.shape[1]))

# 3. Camada SimpleRNN:
# 32: número de unidades (neurônios) na camada recorrente. Este é o tamanho do estado oculto.
modelo_rnn.add(SimpleRNN(32))

# 4. Camada Densa de Saída:
# total_palavras: número de neurônios de saída (um para cada palavra no vocabulário)
# activation='softmax': função de ativação para probabilidade (soma 1 para todas as palavras)
modelo_rnn.add(Dense(total_palavras, activation='softmax'))

# 5. Compilar o modelo
modelo_rnn.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 6. Exibir um resumo da arquitetura do modelo
modelo_rnn.summary()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ simple_rnn (SimpleRNN)          │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

Explicação da Arquitetura:

* Embedding Layer: Essencial em PLN. Ela mapeia cada palavra (representada por seu índice numérico) para um vetor denso de embedding. Este vetor captura relações semânticas entre as palavras. Por exemplo, palavras com significados semelhantes estarão "próximas" no espaço de embedding.
* SimpleRNN Layer: Esta é a camada recorrente. Ela recebe as sequências de embeddings e processa-as passo a passo. O 32 indica a dimensão do vetor de estado oculto (ou seja, a 'memória' que a RNN carrega ao longo do tempo).
* Dense (Output) Layer: Esta camada final recebe o estado oculto final da SimpleRNN e o transforma em um vetor de probabilidades, onde cada posição corresponde a uma palavra do vocabulário. A função softmax garante que a soma dessas probabilidades seja 1.

Passo 4: Treinamento do Modelo

In [ ]:
# 1. Treinando o modelo
print("\nIniciando o treinamento do modelo RNN...")
modelo_rnn.fit(entradas_X_padded, saidas_Y_one_hot, epochs=100, verbose=1)
# epochs: quantas vezes o modelo verá todo o conjunto de dados
# verbose: 1 para mostrar o progresso do treinamento
print("Treinamento concluído!")


Iniciando o treinamento do modelo RNN...
Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.0000e+00 - loss: 3.0023
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.0000e+00 - loss: 2.9938
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step - accuracy: 0.0500 - loss: 2.9853
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.1000 - loss: 2.9767
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1000 - loss: 2.9680
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.1000 - loss: 2.9592
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.1500 - loss: 2.9502
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.2000 - loss: 2.9410
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.2000 - loss: 2.9315
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.2000 - loss: 2.9218
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.2000 - loss: 2.9116
Epoch 12/100
1/1 ━━━━━━━━━

Passo 5: Usar o Modelo para Previsão

In [ ]:
def prever_proxima_palavra(modelo, tokenizer, max_seq_len, texto_base):
    """
    Prevê a próxima palavra dado um texto base.
    """
    # Converter o texto base para sequência numérica
    sequencia_numerica = tokenizer.texts_to_sequences([texto_base])[0]

    # Padronizar o comprimento da sequência de entrada (precisa ter o mesmo formato que o treinamento)
    # Atenção: max_seq_len deve ser o comprimento que as "entradas" foram pad_sequences
    sequencia_padded = pad_sequences([sequencia_numerica], maxlen=max_seq_len, padding='pre')

    # Fazer a previsão
    previsao_probabilidades = modelo.predict(sequencia_padded, verbose=0)[0]

    # Obter o índice da palavra com a maior probabilidade
    indice_palavra_prevista = np.argmax(previsao_probabilidades)

    # Converter o índice de volta para a palavra
    for palavra, indice in tokenizer.word_index.items():
        if indice == indice_palavra_prevista:
            return palavra
    return None # Caso a palavra não seja encontrada (improvável com o vocabulário ajustado)

# Comprimento de entrada esperado pelo modelo
# entradas_X_padded.shape[1] É o maxlen que usamos no pad_sequences para X
comprimento_entrada_modelo = entradas_X_padded.shape[1]

# Testar o modelo com novas frases
print("\n--- Testando o Modelo RNN ---")

texto_teste_1 = "eu gosto de"
proxima_1 = prever_proxima_palavra(modelo_rnn, tokenizer, comprimento_entrada_modelo, texto_teste_1)
print(f"Texto: '{texto_teste_1}' -> Próxima palavra prevista: '{proxima_1}'")

texto_teste_2 = "python é uma"
proxima_2 = prever_proxima_palavra(modelo_rnn, tokenizer, comprimento_entrada_modelo, texto_teste_2)
print(f"Texto: '{texto_teste_2}' -> Próxima palavra prevista: '{proxima_2}'")

texto_teste_3 = "programar é divertido"
proxima_3 = prever_proxima_palavra(modelo_rnn, tokenizer, comprimento_entrada_modelo, texto_teste_3)
print(f"Texto: '{texto_teste_3}' -> Próxima palavra prevista: '{proxima_3}'")

texto_teste_4 = "aprenda python e"
proxima_4 = prever_proxima_palavra(modelo_rnn, tokenizer, comprimento_entrada_modelo, texto_teste_4)
print(f"Texto: '{texto_teste_4}' -> Próxima palavra prevista: '{proxima_4}'")

# Exemplo com palavra fora do vocabulário (ou sequência que o modelo nunca viu antes)
texto_teste_5 = "o sol brilha no" # Palavras "sol" e "brilha" não estão no vocabulário
proxima_5 = prever_proxima_palavra(modelo_rnn, tokenizer, comprimento_entrada_modelo, texto_teste_5)
print(f"Texto: '{texto_teste_5}' -> Próxima palavra prevista: '{proxima_5}' (Pode ser inesperada devido a palavras desconhecidas)")


--- Testando o Modelo RNN ---
Texto: 'eu gosto de' -> Próxima palavra prevista: 'programar'
Texto: 'python é uma' -> Próxima palavra prevista: 'linguagem'
Texto: 'programar é divertido' -> Próxima palavra prevista: 'de'
Texto: 'aprenda python e' -> Próxima palavra prevista: 'seja'
Texto: 'o sol brilha no' -> Próxima palavra prevista: 'de' (Pode ser inesperada devido a palavras desconhecidas)
